In [ ]:
#今の所最適なプログラム
#動画に対して処理
import cv2
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import io

resized_size = (640, 480)
def SearchCF(params, d, theta_cal, r_obs):
    a, b, c = params
    r_cal = []
    for i, _ in enumerate(theta_cal):
        theta_prime = (d / 3 * theta_cal[i] - c) % (np.pi / 1.5)
        r_cal.append(a * (b * (1 / np.cos(theta_prime - np.pi / 3) - 2) + 2))
    
    MSE = np.mean((np.array(r_obs) - np.array(r_cal)) ** 2 / np.array(r_cal) ** 2)
    return MSE

def drawContour(params, d, ret):
    a, b, c = params
    r = []
    theta = np.linspace(0, 2 * np.pi, 1000)
    for i, _ in enumerate(theta):
        theta_prime = (d / 3 * theta[i] - c) % (np.pi / 1.5)
        r.append(a * (b * (1 / np.cos(theta_prime - np.pi / 3) - 2) + 2))
    
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(1, 1, 1)
    plt.rcParams["font.family"] = "Arial"
    ax.plot(x, y)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    textstr = '\n'.join((
        r'$a=%.2f$' % (a,),
        r'$b=%.2f$' % (b,),
        r'$c=%.2f$' % (c,),
        r'$d=%.2f$' % (d,),
        r'$r=a(b(sec(θ - π/3)-2)+2)$'
    ))
    
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
    ax.set_xlim([-90, 90])
    ax.set_ylim([-60, 60])
    ax.invert_yaxis()
    #plt.tight_layout()
    # 画像保存前にリサイズ
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img = cv2.imdecode(np.frombuffer(buf.read(), np.uint8), 1)
    buf.close()
    
    # 画像を指定サイズにリサイズ
    img_resized = cv2.resize(img, resized_size)
    
    # リサイズした画像を保存
    cv2.imwrite("/Users/nametakouhei/name/画像ファイル/保存データ/四面体/Predicted_Contour/image_{}.jpg".format(frame_number), img_resized)
    
    plt.show()
    plt.close()

# main関数
cap = cv2.VideoCapture('/Users/nametakouhei/name/画像ファイル/四面体.mp4')
d = None
minimum_value = []
frame_number = 0
for _ in range(25):
    ret, frame = cap.read()
    frame_number += 1 
    if not ret or frame is None:
        print("Error: Failed to load image from the video")
        sys.exit(1)

    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.bilateralFilter(gray_image, 7, 15, 15)
    denoised_image = cv2.medianBlur(blurred_image, 5)
    ret, binary = cv2.threshold(denoised_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    conditions = {"min_contour_area": 2500, "max_contour_area": 9000}
    center_x, center_y = gray_image.shape[1] // 2, gray_image.shape[0] // 2  
    max_distance_from_center = 150  # 中央からの最大距離
    
    filtered_contours = []
    moment_data = []
    for _, contour in enumerate(contours):
        contour_area = cv2.contourArea(contour)
        M = cv2.moments(contour)
        filtered_cx = int(M['m10'] / M['m00']) if M["m00"] != 0 else 0
        filtered_cy = int(M['m01'] / M['m00']) if M["m00"] != 0 else 0
        distance_from_center = np.sqrt((filtered_cx - center_x)**2 + (filtered_cy - center_y)**2)
        
        if (
            ("min_contour_area" not in conditions or contour_area >= conditions["min_contour_area"]) and
            ("max_contour_area" not in conditions or contour_area <= conditions["max_contour_area"]) and
            (distance_from_center <= max_distance_from_center)
        ):
            filtered_contours.append(contour)
            moment_data.append((filtered_cx, filtered_cy))
    
    result_image = frame.copy()
    MSE_standard = float('inf')
    params_result = [0, 0, 0]
    if d == None:
        for contour, moment in zip(filtered_contours, moment_data):
            M = cv2.moments(contour)
            rel_con = []
            if M["m00"] != 0:
                filtered_cx, filtered_cy = moment
                cv2.circle(result_image, (filtered_cx, filtered_cy), 5, (0, 0, 255), -1)
                cv2.drawContours(result_image, [contour], -1, (0, 255, 0), 2)
                rel_con = contour - moment

            con_x = []
            con_y = []

            for i, con in enumerate(rel_con):
                con_x.append(con[0][0])
                con_y.append(con[0][1])

            r_obs = [np.sqrt(x**2 + y**2) for x, y in zip(con_x, con_y)]
            theta_obs = [np.arctan2(y, x) for x, y in zip(con_x, con_y)]

            for i in range(5):
                initial_guess = [21, 0.8, 1.5]
                result = minimize(SearchCF, initial_guess, args=(i, theta_obs, r_obs), method='Nelder-Mead', options={'disp': True})
                
                print("Optimal parameters:", result.x)
                print("Minimum value:", result.fun)

                if result.fun < MSE_standard:
                    MSE_standard = result.fun
                    params_result = result.x
                    d = i
                    minimum_value.append(result.fun)

            drawContour(params_result,d,frame_number)

    else:
        for contour, moment in zip(filtered_contours, moment_data):
                M = cv2.moments(contour)
                rel_con = []

                if M["m00"] != 0:
                    filtered_cx, filtered_cy = moment
                    cv2.circle(result_image, (filtered_cx, filtered_cy), 5, (0, 0, 255), -1)
                    cv2.drawContours(result_image, [contour], -1, (0, 255, 0), 2)
                    rel_con = contour - moment

                con_x = []
                con_y = []

                for i, con in enumerate(rel_con):
                    con_x.append(con[0][0])
                    con_y.append(con[0][1])

                r_obs = [np.sqrt(x**2 + y**2) for x, y in zip(con_x, con_y)]
                theta_obs = [np.arctan2(y, x) for x, y in zip(con_x, con_y)]

                initial_guess = [22, 0.4, 1.5]
                result = minimize(SearchCF, initial_guess, args=(i, theta_obs, r_obs), method='BFGS', options={'disp': True})
                
                print("Optimal parameters:", result.x)
                print("Minimum value:", result.fun)
        drawContour(result.x,d,frame_number)
        minimum_value.append(result.fun)
        plt.imshow(result_image)
        plt.title('Processed Image')
        plt.savefig("/Users/nametakouhei/name/画像ファイル/保存データ/四面体/Processed_image/image_{}.png".format(frame_number))
        plt.show()
        plt.close()

print(minimum_value)
t = np.arange(0,len(minimum_value),1)
plt.plot(t,minimum_value)
plt.xlabel("time / [frame]")
plt.ylabel("MSE / [-]")
plt.show()